# MMLU Evaluation of LLaMA 3.2 1B Evaluation

In [1]:
import os
# no nvlink
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
# use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
from transformers import LlamaForCausalLM, AutoTokenizer

print("Torch version:", torch.__version__)
print("Is CUDA available?", torch.cuda.is_available())

Torch version: 2.4.1+cu121
Is CUDA available? True


In [3]:
model_path = "/data/llm/llama/Llama-3.2-1B-Instruct/"

model = LlamaForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()
    

In [4]:
from datasets import load_dataset

# Load MMLU dataset
dataset = load_dataset("cais/mmlu", "all", split="test")

In [5]:
print(len(dataset))

14042


# Run 1: 2 shot on simple questions

Questions are defined by me, questions that are way too outmatched by the complexity of the task of MMLU.

In [6]:
from transformers import pipeline

pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype = torch.bfloat16,
    device_map='auto',
    pad_token_id=tokenizer.eos_token_id,
)

example = dataset[300]
question = example['question']
choices = example['choices']
correct_answer = example['answer']


initial_prompt = '''
Please answer the following user's question in the format "Answer: ANSWER_TO_THE_QUESTION.".

For example, user input would be:

Question: What is 2 + 2?
Choices: ['1', '2', '3', '4']

Your answer would be:
Answer: 4

Another example would be:

Question: Which one of these parts are used for seeing?
Choices: ['eyes', 'nose', 'ears', 'mouth']

Your answer would be:
Answer: eyes
'''


user_question = 'Question: ' + question + "\nChoices: " + str(choices)

# question_prompt = input_text
print('Initial prompt: ', initial_prompt)
print('User question: ', user_question)
print('Choices: ', choices)

messages = [
    {'role': 'system', 'content': initial_prompt},
    {'role': 'user', 'content': user_question},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)

output = outputs[0]['generated_text'][-1]['content']
print('LLaMA 3.21B response: ', output)
print('Correct answer: ', choices[correct_answer])
print(choices[correct_answer].lower() in output.lower())


Initial prompt:  
Please answer the following user's question in the format "Answer: ANSWER_TO_THE_QUESTION.".

For example, user input would be:

Question: What is 2 + 2?
Choices: ['1', '2', '3', '4']

Your answer would be:
Answer: 4

Another example would be:

Question: Which one of these parts are used for seeing?
Choices: ['eyes', 'nose', 'ears', 'mouth']

Your answer would be:
Answer: eyes

User question:  Question: Which is not a similarity between Saturn and Jupiter's atmospheres?
Choices: ['a composition dominated by hydrogen and helium', 'the presence of belts zones and storms', 'an equatorial wind speed of more than 900 miles per hour', 'significant "shear" between bands of circulation at different latitudes']
Choices:  ['a composition dominated by hydrogen and helium', 'the presence of belts zones and storms', 'an equatorial wind speed of more than 900 miles per hour', 'significant "shear" between bands of circulation at different latitudes']


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


LLaMA 3.21B response:  Answer: ANSWER_TO_THE_question.

Explanation: 
- Saturn's atmosphere is primarily composed of hydrogen and helium.
- Jupiter's atmosphere features belts zones and storms.
- An equatorial wind speed of more than 900 miles per hour is a characteristic of Jupiter's atmosphere.
- Significant "shear" between bands of circulation at different latitudes is a characteristic of Saturn's atmosphere.

Saturn's atmosphere is different from Jupiter's, as it is primarily composed of hydrogen and helium, whereas Jupiter's atmosphere is dominated by hydrogen and helium.
Correct answer:  an equatorial wind speed of more than 900 miles per hour
True


In [8]:
from tqdm.notebook import tqdm
import csv
from transformers import pipeline

pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype = torch.bfloat16,
    device_map='auto',
    pad_token_id=tokenizer.eos_token_id,
)

# Assuming the dataset and pipeline are already set up as per your current workflow
output_file = 'llama_mmlu_responses.csv'

# Sample dataset size, you can adjust it as needed for the loop
dataset_size = len(dataset)  # Assuming dataset is loaded with all examples

# Initialize a list to collect all rows
rows = []

# Loop through the dataset and gather question, choices, and answers
for idx in tqdm(range(dataset_size)):
    example = dataset[idx]
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']
    
    initial_prompt = '''
    Please answer the following user's question in the format "Answer: ANSWER_TO_THE_QUESTION.".

    For example, user input would be:

    Question: What is 2 + 2?
    Choices: ['1', '2', '3', '4']

    Your answer would be:
    Answer: 4

    Another example would be:

    Question: Which one of these parts are used for seeing?
    Choices: ['eyes', 'nose', 'ears', 'mouth']

    Your answer would be:
    Answer: eyes
    '''

    user_question = 'Question: ' + question + "\nChoices: " + str(choices)

    # Generating messages for the LLaMA model
    messages = [
        {'role': 'system', 'content': initial_prompt},
        {'role': 'user', 'content': user_question},
    ]

    # Get LLaMA's response
    outputs = pipe(messages, max_new_tokens=10)
    output = outputs[0]['generated_text'][-1]['content']

    # Extract LLaMA's answer (post-processed)
    answer_start = output.lower().find("answer:") + len("answer:")
    llama_answer = output[answer_start:].strip().split()[0]  # Taking the first word as the final answer

    # Checking if LLaMA's answer matches the correct answer
    is_correct = choices[correct_answer].lower() in output.lower()

    # Collect the row
    rows.append([question, choices, llama_answer, choices[correct_answer], is_correct])

# Write all rows to the CSV file in one go
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")


  0%|          | 0/14042 [00:00<?, ?it/s]

Results saved to llama_mmlu_responses.csv


In [9]:
import pandas as pd

df = pd.read_csv('llama_mmlu_responses_run1.csv')
print(f'Average: {float(df["Is Correct"].mean()) * 100:.1f}')

Average: 21.3


# Run 2: 5-shot on samples taken from MMLU, change format to JSON

Randomly sample 5 items from the MMLU dataset, without caring for alignment of the subjects of the samples to the subject of the question. Format response in JSON

In [10]:
from datasets import load_dataset

# Load MMLU dataset
train_dataset = load_dataset("cais/mmlu", "all", split="dev")

In [11]:
import random

n_shots = 5
train_indices = random.sample(range(len(train_dataset)), n_shots)  # Get random indices
few_shot_examples = train_dataset.select(train_indices)  # Select the few-shot examples

for example in few_shot_examples:
    question = example['question']
    choices = example['choices']
    correct_answer = choices[example['answer']]
    print(question, choices, correct_answer)

Morning sickness is typically a problem: ['during the first trimester', 'during the second trimester', 'during the third trimester', 'all through the pregnancy'] during the first trimester
The population of the city where Michelle was born is 145,826. What is the value of the 5 in the number 145,826? ['5 thousands', '5 hundreds', '5 tens', '5 ones'] 5 thousands
Which element in tobacco smoke is responsible for cancers? ['Nicotine', 'Tar', 'Carbon monoxide', 'Smoke particles'] Tar
What is the embryological origin of the hyoid bone? ['The first pharyngeal arch', 'The first and second pharyngeal arches', 'The second pharyngeal arch', 'The second and third pharyngeal arches'] The second and third pharyngeal arches
3 Cl−(aq) + 4 CrO_4^2−(aq) + 23 H+(aq) → 3 HClO2(aq) + 4 Cr3+(aq) + 10 H2O(l). In the reaction shown above, Cl−(aq) behaves as ['an acid', 'a base', 'a catalyst', 'a reducing agent'] a reducing agent


In [14]:
import csv
from tqdm.notebook import tqdm
from transformers import pipeline
import torch

# Load your dataset and set up the model pipeline
output_file = 'llama_mmlu_responses_run2.csv'

# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your LLaMA model pipeline
pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype=torch.float16,  # Use FP16 for faster GPU performance
    device=0,  # Use the first GPU
    pad_token_id=tokenizer.eos_token_id
)

# Sample n_shots indices from the training set
train_indices = random.sample(range(len(train_dataset)), n_shots)  # Get random indices
few_shot_examples = train_dataset.select(train_indices)  # Select the few-shot examples

# Create the few-shot prompt

few_shot_prompt = '''
You are a bot that only answers the following question in JSON format.

For example, if the question is:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']

Your answer would be:
{"Answer": "Dove symbol"}

--------------------------

Another example question is:

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']

And your answer would be:
{"Answer": "5.78"}

--------------------------

Another example question is:

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl is embarrassed to live in a trailer park, so I invite the class to her house.
Choices: ['Wrong, Wrong', 'Wrong, Not wrong', 'Not wrong, Wrong', 'Not wrong, Not wrong']

And your answer would be:
{"Answer": "Not wrong, Wrong"}

--------------------------

Another example question is:

Question: As of 2015, since 1990 forests have ____ in Europe and have ____ in Africa and the Americas.
Choices:  ['increased, increased', 'increased, decreased', 'decreased, increased', 'decreased, decreased']

And your answer would be:
{"Answer": "increased, decreased"}

--------------------------

Another example question is:

Question: In the program below, the initial value of x is 5 and the initial value of y is 10.
 IF (X < O)
 {
 DISPLAY ("Foxtrot")
 }
 ELSE
 {
 IF (X > y)
 {
   DISPLAY ("Hotel")
 }
 ELSE 
 {
   IF (y > O)
   {
   DISPLAY ("November")
   }
   ELSE
   {
   DISPLAY ("Yankee")
   }
 }
 }
What is displayed as a result of running the program?
Choices: ['Foxtrot', 'Hotel', 'November', 'Yankee']

And your answer would be:
{"Answer": "November"}

'''
# for example in few_shot_examples:
#     question = example['question']
#     choices = example['choices']
#     correct_answer = choices[example['answer']]
    
#     few_shot_prompt += f"Question: {question}\nChoices: {choices}\nAnswer: {correct_answer}\n\n"

print(few_shot_prompt)
# Initialize a list to collect all rows


You are a bot that only answers the following question in JSON format.

For example, if the question is:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']

Your answer would be:
{"Answer": "Dove symbol"}

--------------------------

Another example question is:

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']

And your answer would be:
{"Answer": "5.78"}

--------------------------

Another example question is:

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl

In [15]:
import json
import re

# Processing the evaluation set with tqdm for a progress bar
rows = []
is_correct_count = 0

for idx in tqdm(range(0, len(dataset)), desc="Processing evaluation set"):
    format_error = 'None'
    example = dataset[idx]

    # Prepare the user question with few-shot prompt
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']
    
    stringified_choices = re.sub('"', '\'', str(choices))
    # Append the new question to the few-shot prompt
    user_question = f"Question: {question}\nChoices: {stringified_choices}\n"
    
    # Messages to be passed to the model
    messages = [
        {'role': 'system', 'content': few_shot_prompt},
        {'role': 'user', 'content': user_question},
    ]
    
    # Run the model with torch.cuda.amp for mixed precision (parallel processing)
    outputs = pipe(messages, max_new_tokens=10)
    
    output = outputs[0]['generated_text'][-1]['content']
    
    # Extract LLaMA's answer (post-processed)
    llama_answer = output.lower()
    try:
        llama_answer = json.loads(llama_answer)['answer']
    except Exception as e:
        format_error = 'Answer is not in JSON format.'


    # Check if the answer matches the correct answer
    if type(llama_answer) == str:
        is_correct = choices[correct_answer].lower() == re.sub('"', '\'', llama_answer).lower()
    else:
        is_correct = False
        format_error = 'Parsed answer is still dictionary.'

    is_correct_count += 1 if is_correct else 0
    if idx % 100 == 0:
        print(f'{is_correct_count} / {idx} Correct Answers')
    # Collect the result row
    rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, format_error])

# Write the collected results to a CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct", "Format Error"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")

Processing evaluation set:   0%|          | 0/14042 [00:00<?, ?it/s]

0 / 0 Correct Answers
11 / 100 Correct Answers
38 / 200 Correct Answers
53 / 300 Correct Answers
69 / 400 Correct Answers
89 / 500 Correct Answers
100 / 600 Correct Answers
113 / 700 Correct Answers
127 / 800 Correct Answers
137 / 900 Correct Answers
153 / 1000 Correct Answers
172 / 1100 Correct Answers
184 / 1200 Correct Answers
195 / 1300 Correct Answers
212 / 1400 Correct Answers
231 / 1500 Correct Answers
257 / 1600 Correct Answers
284 / 1700 Correct Answers
306 / 1800 Correct Answers
311 / 1900 Correct Answers
331 / 2000 Correct Answers
356 / 2100 Correct Answers
378 / 2200 Correct Answers
396 / 2300 Correct Answers
419 / 2400 Correct Answers
434 / 2500 Correct Answers
443 / 2600 Correct Answers
473 / 2700 Correct Answers
485 / 2800 Correct Answers
496 / 2900 Correct Answers
511 / 3000 Correct Answers
517 / 3100 Correct Answers
530 / 3200 Correct Answers
543 / 3300 Correct Answers
549 / 3400 Correct Answers
572 / 3500 Correct Answers
597 / 3600 Correct Answers
617 / 3700 Correct A

In [17]:
import pandas as pd

df = pd.read_csv('llama_mmlu_responses_run2.csv')

print(f'Average: {float(df["Is Correct"].mean()) * 100}')

Average: 15.71713431135166


# Run 3: Try fixing the temperature

Temperature set at 0.1 for deterministic responses.

In [51]:
import csv
from tqdm.notebook import tqdm
from transformers import pipeline
import torch

# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your LLaMA model pipeline
pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype=torch.float16,  # Use FP16 for faster GPU performance
    device=0,  # Use the first GPU
    pad_token_id=tokenizer.eos_token_id
)

# Create the few-shot prompt
low_temp_few_shot_prompt = '''
You are a bot that only answers the following question in JSON format.

For example, if the question is:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']

Your answer would be:
{"Answer": "Dove symbol"}

--------------------------

Another example question is:

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']

And your answer would be:
{"Answer": "5.78"}

--------------------------

Another example question is:

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl is embarrassed to live in a trailer park, so I invite the class to her house.
Choices: ['Wrong, Wrong', 'Wrong, Not wrong', 'Not wrong, Wrong', 'Not wrong, Not wrong']

And your answer would be:
{"Answer": "Not wrong, Wrong"}

--------------------------

Another example question is:

Question: As of 2015, since 1990 forests have ____ in Europe and have ____ in Africa and the Americas.
Choices:  ['increased, increased', 'increased, decreased', 'decreased, increased', 'decreased, decreased']

And your answer would be:
{"Answer": "increased, decreased"}

--------------------------

Another example question is:

Question: In the program below, the initial value of x is 5 and the initial value of y is 10.
 IF (X < O)
 {
 DISPLAY ("Foxtrot")
 }
 ELSE
 {
 IF (X > y)
 {
   DISPLAY ("Hotel")
 }
 ELSE 
 {
   IF (y > O)
   {
   DISPLAY ("November")
   }
   ELSE
   {
   DISPLAY ("Yankee")
   }
 }
 }
What is displayed as a result of running the program?
Choices: ['Foxtrot', 'Hotel', 'November', 'Yankee']

And your answer would be:
{"Answer": "November"}
'''

print(few_shot_prompt)


You are a bot that only answers the following question in JSON format.

For example, if the question is:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']

Your answer would be:
{"Answer": "Dove symbol"}

--------------------------

Another example question is:

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']

And your answer would be:
{"Answer": "5.78"}

--------------------------

Another example question is:

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl

In [62]:
import json
import re

# Processing the evaluation set with tqdm for a progress bar
rows = []
is_correct_count = 0
output_file = 'llama_mmlu_responses_run3.csv'

for idx in tqdm(range(0, len(dataset)), desc="Processing evaluation set"):
    format_error = 'None'
    example = dataset[idx]

    # Prepare the user question with few-shot prompt
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']
    
    stringified_choices = re.sub('"', '\'', str(choices))
    # Append the new question to the few-shot prompt
    user_question = f"Question: {question}\nChoices: {stringified_choices}\n"
    
    # Messages to be passed to the model
    messages = [
        {'role': 'system', 'content': low_temp_few_shot_prompt},
        {'role': 'user', 'content': user_question},
    ]
    
    # Run the model with torch.cuda.amp for mixed precision (parallel processing)
    outputs = pipe(messages, max_new_tokens=256, temperature=0.1)
    
    output = outputs[0]['generated_text'][-1]['content']
    
    # Extract LLaMA's answer (post-processed)
    llama_answer = output.lower()
    try:
        llama_answer = json.loads(llama_answer)['answer']
    except Exception as e:
        format_error = 'Answer is not in JSON format.'


    # Check if the answer matches the correct answer
    if type(llama_answer) == str:
        is_correct = choices[correct_answer].lower() == re.sub('"', '\'', llama_answer).lower()
    else:
        is_correct = False
        format_error = 'Parsed answer is still dictionary.'

    is_correct_count += 1 if is_correct else 0
    if idx % 100 == 0:
        print(f'{is_correct_count} / {idx} Correct Answers')
    # Collect the result row
    rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, format_error])

# Write the collected results to a CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct", "Format Error"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")

Processing evaluation set:   0%|          | 0/14042 [00:00<?, ?it/s]

0 / 0 Correct Answers
19 / 100 Correct Answers
62 / 200 Correct Answers
98 / 300 Correct Answers
131 / 400 Correct Answers
164 / 500 Correct Answers
195 / 600 Correct Answers
226 / 700 Correct Answers
268 / 800 Correct Answers
295 / 900 Correct Answers
315 / 1000 Correct Answers
330 / 1100 Correct Answers
344 / 1200 Correct Answers
377 / 1300 Correct Answers
405 / 1400 Correct Answers
426 / 1500 Correct Answers
460 / 1600 Correct Answers
490 / 1700 Correct Answers
522 / 1800 Correct Answers
535 / 1900 Correct Answers
569 / 2000 Correct Answers
603 / 2100 Correct Answers
628 / 2200 Correct Answers
659 / 2300 Correct Answers
683 / 2400 Correct Answers
708 / 2500 Correct Answers
730 / 2600 Correct Answers
763 / 2700 Correct Answers
800 / 2800 Correct Answers
832 / 2900 Correct Answers
860 / 3000 Correct Answers
886 / 3100 Correct Answers
918 / 3200 Correct Answers
949 / 3300 Correct Answers
982 / 3400 Correct Answers
1023 / 3500 Correct Answers
1062 / 3600 Correct Answers
1099 / 3700 Corr

In [18]:
import pandas as pd

df = pd.read_csv('llama_mmlu_responses_run3.csv')

print(f'Average: {float(df["Is Correct"].mean()) * 100}')

Average: 30.4871100982766


## Where the discrepancies might come from

1. Perhaps increase max tokens generated in prompt?
2. Perhaps simplify the prompt even further?
3. Perhaps compartmentalize the querying of the model on each subject?

# Run 4: Increase tokens in generated result

Set to max_token=256.


In [37]:
import csv
from tqdm.notebook import tqdm
from transformers import pipeline
import torch

# Load your dataset and set up the model pipeline
# output_file = 'llama_mmlu_responses_5shot.csv'

# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your LLaMA model pipeline
pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype=torch.float16,  # Use FP16 for faster GPU performance
    device=0,  # Use the first GPU
    pad_token_id=tokenizer.eos_token_id
)

# Sample n_shots indices from the training set
# train_indices = random.sample(range(len(train_dataset)), n_shots)  # Get random indices
# few_shot_examples = train_dataset.select(train_indices)  # Select the few-shot examples

# Create the few-shot prompt

few_shot_prompt = '''
You are a bot that only answers the following question in JSON format.

For example, if the question is:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']

Your answer would be:
{"Answer": "Dove symbol"}

--------------------------

Another example question is:

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']

And your answer would be:
{"Answer": "5.78"}

--------------------------

Another example question is:

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl is embarrassed to live in a trailer park, so I invite the class to her house.
Choices: ['Wrong, Wrong', 'Wrong, Not wrong', 'Not wrong, Wrong', 'Not wrong, Not wrong']

And your answer would be:
{"Answer": "Not wrong, Wrong"}

--------------------------

Another example question is:

Question: As of 2015, since 1990 forests have ____ in Europe and have ____ in Africa and the Americas.
Choices:  ['increased, increased', 'increased, decreased', 'decreased, increased', 'decreased, decreased']

And your answer would be:
{"Answer": "increased, decreased"}

--------------------------

Another example question is:

Question: In the program below, the initial value of x is 5 and the initial value of y is 10.
 IF (X < O)
 {
 DISPLAY ("Foxtrot")
 }
 ELSE
 {
 IF (X > y)
 {
   DISPLAY ("Hotel")
 }
 ELSE 
 {
   IF (y > O)
   {
   DISPLAY ("November")
   }
   ELSE
   {
   DISPLAY ("Yankee")
   }
 }
 }
What is displayed as a result of running the program?
Choices: ['Foxtrot', 'Hotel', 'November', 'Yankee']

And your answer would be:
{"Answer": "November"}
'''
# for example in few_shot_examples:
#     question = example['question']
#     choices = example['choices']
#     correct_answer = choices[example['answer']]
    
#     few_shot_prompt += f"Question: {question}\nChoices: {choices}\nAnswer: {correct_answer}\n\n"

print(few_shot_prompt)
# Initialize a list to collect all rows


You are a bot that only answers the following question in JSON format.

For example, if the question is:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']

Your answer would be:
{"Answer": "Dove symbol"}

--------------------------

Another example question is:

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']

And your answer would be:
{"Answer": "5.78"}

--------------------------

Another example question is:

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl

In [39]:
import json
import re
from tqdm.notebook import tqdm

# Processing the evaluation set with tqdm for a progress bar
rows = []
is_correct_count = 0
output_file = 'llama_mmlu_responses_run4.csv'

for idx in tqdm(range(0, len(train_dataset)), desc="Processing evaluation set"):
    format_error = 'None'
    example = dataset[idx]

    # Prepare the user question with few-shot prompt
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']
    
    stringified_choices = re.sub('"', '\'', str(choices))
    # Append the new question to the few-shot prompt
    user_question = f"Question: {question}\nChoices: {stringified_choices}\n"
    
    # Messages to be passed to the model
    messages = [
        {'role': 'system', 'content': few_shot_prompt},
        {'role': 'user', 'content': user_question},
    ]
    outputs = pipe(messages, max_new_tokens=256)
    output = outputs[0]['generated_text'][-1]['content']
    
    # Extract LLaMA's answer (post-processed)
    llama_answer = output.lower()
    try:
        llama_answer = json.loads(llama_answer)['answer']
    except Exception as e:
        format_error = 'Answer is not in JSON format.'


    # Check if the answer matches the correct answer
    if type(llama_answer) == str:
        is_correct = choices[correct_answer].lower() == re.sub('"', '\'', llama_answer).lower()
    else:
        is_correct = False
        format_error = 'Parsed answer is still dictionary.'

    is_correct_count += 1 if is_correct else 0
    if idx % 100 == 0:
        print(f'{is_correct_count} / {idx} Correct Answers')
    # Collect the result row
    rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, format_error])

# Write the collected results to a CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct", "Format Error"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")

Processing evaluation set:   0%|          | 0/285 [00:00<?, ?it/s]

1 / 0 Correct Answers
19 / 100 Correct Answers
69 / 200 Correct Answers
Results saved to llama_mmlu_responses_run5.csv


In [21]:
import pandas as pd

df = pd.read_csv('llama_mmlu_responses_run4.csv')

print(f'Average: {float(df["Is Correct"].mean()) * 100}')

Average: 35.78947368421053


# Run 5: Use a more directed 5-shot prompt

Removed some extra spaces and newlines. End with a final instruction: "After here, you will be given questions. Select the best answer from the choices provided. Do not explain your answer. Format your answer as with the examples above."

In [65]:
import csv
from tqdm.notebook import tqdm
from transformers import pipeline
import torch

# Load your dataset and set up the model pipeline
# output_file = 'llama_mmlu_responses_5shot.csv'

# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your LLaMA model pipeline
pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype=torch.float16,  # Use FP16 for faster GPU performance
    device=0,  # Use the first GPU
    pad_token_id=tokenizer.eos_token_id
)

# Sample n_shots indices from the training set
# train_indices = random.sample(range(len(train_dataset)), n_shots)  # Get random indices
# few_shot_examples = train_dataset.select(train_indices)  # Select the few-shot examples

# Create the few-shot prompt

few_shot_prompt = '''
You are a bot that's trained to answer questions in JSON format. For example:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']
{"Answer": "Dove symbol"}

--------------------------

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']
{"Answer": 5.78}

--------------------------

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl is embarrassed to live in a trailer park, so I invite the class to her house.
Choices: ['Wrong, Wrong', 'Wrong, Not wrong', 'Not wrong, Wrong', 'Not wrong, Not wrong']
{"Answer": "Not wrong, Wrong"}

--------------------------

Question: As of 2015, since 1990 forests have ____ in Europe and have ____ in Africa and the Americas.
Choices:  ['increased, increased', 'increased, decreased', 'decreased, increased', 'decreased, decreased']
{"Answer": "increased, decreased"}

--------------------------

Question: In the program below, the initial value of x is 5 and the initial value of y is 10.
 IF (X < O)
 {
 DISPLAY ("Foxtrot")
 }
 ELSE
 {
 IF (X > y)
 {
   DISPLAY ("Hotel")
 }
 ELSE 
 {
   IF (y > O)
   {
   DISPLAY ("November")
   }
   ELSE
   {
   DISPLAY ("Yankee")
   }
 }
 }
What is displayed as a result of running the program?
Choices: ['Foxtrot', 'Hotel', 'November', 'Yankee']
{"Answer": "November"}

After here, you will be given questions. Select the best answer from the choices provided. Do not explain your answer. Format your answer as with the examples above.
'''

print(few_shot_prompt)


You are a bot that's trained to answer questions in JSON format. For example:

Question: Which one of the following items is an example of nonmaterial culture?
Choices: ['Dove soap', 'Dove candy bar', 'Dove symbol', 'A dove (bird)']
{"Answer": "Dove symbol"}

--------------------------

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
Choices: ['3.89', '7.78', '5.78', '2.33']
{"Answer": 5.78}

--------------------------

Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I really loved my newborn baby boy. Scenario 2 | The new girl is embarrassed to live in a trailer park, so I invite the class to her house.
Choices: ['Wrong, Wrong', 'Wrong, Not wrong', 'Not wrong, Wr

In [89]:
import json
import re
from tqdm.notebook import tqdm

# Processing the evaluation set with tqdm for a progress bar
rows = []
is_correct_count = 0
output_file = 'llama_mmlu_responses_run5.csv'

for idx in tqdm(range(0, len(dataset)), desc="Processing evaluation set"):
    format_error = 'None'
    example = dataset[idx]

    # Prepare the user question with few-shot prompt
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']
    
    stringified_choices = re.sub('"', '\'', str(choices))
    # Append the new question to the few-shot prompt
    user_question = f"Question: {question}\nChoices: {stringified_choices}\nWhat is your final answer?"
    
    # Messages to be passed to the model
    messages = [
        {'role': 'system', 'content': few_shot_prompt},
        {'role': 'user', 'content': user_question},
    ]
    
    # Run the model with torch.cuda.amp for mixed precision (parallel processing)
    outputs = pipe(messages, max_new_tokens=64, temperature=0.1)
    
    output = outputs[0]['generated_text'][-1]['content']
    
    # Extract LLaMA's answer (post-processed)
    llama_answer = output.lower()
    try:
        llama_answer = json.loads(llama_answer)['answer']
    except Exception as e:
        format_error = 'Answer is not in JSON format.'
        is_correct = choices[correct_answer].lower() in re.sub('"', '\'', llama_answer).lower()
        is_correct_count += 1 if is_correct else 0
        rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, format_error])
        continue


    # Check if the answer matches the correct answer
    if type(llama_answer) == str:
        is_correct = choices[correct_answer].lower() == re.sub('"', '\'', llama_answer).lower()
    else:
        is_correct = False
        format_error = 'Parsed answer is still dictionary.'

    is_correct_count += 1 if is_correct else 0
    if idx % 100 == 0:
        print(f'{is_correct_count} / {idx} Correct Answers')
    # Collect the result row
    rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, format_error])

# Write the collected results to a CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct", "Format Error"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")

Processing evaluation set:   0%|          | 0/14042 [00:00<?, ?it/s]

0 / 0 Correct Answers
20 / 100 Correct Answers
64 / 200 Correct Answers
105 / 300 Correct Answers
143 / 400 Correct Answers
185 / 500 Correct Answers
218 / 600 Correct Answers
262 / 700 Correct Answers
306 / 800 Correct Answers
339 / 900 Correct Answers
364 / 1000 Correct Answers
425 / 1200 Correct Answers
462 / 1300 Correct Answers
494 / 1400 Correct Answers
524 / 1500 Correct Answers
569 / 1600 Correct Answers
608 / 1700 Correct Answers
645 / 1800 Correct Answers
665 / 1900 Correct Answers
708 / 2000 Correct Answers
753 / 2100 Correct Answers
786 / 2200 Correct Answers
825 / 2300 Correct Answers
854 / 2400 Correct Answers
885 / 2500 Correct Answers
915 / 2600 Correct Answers
956 / 2700 Correct Answers
1000 / 2800 Correct Answers
1080 / 3000 Correct Answers
1147 / 3200 Correct Answers
1185 / 3300 Correct Answers
1227 / 3400 Correct Answers
1268 / 3500 Correct Answers
1310 / 3600 Correct Answers
1351 / 3700 Correct Answers
1398 / 3800 Correct Answers
1432 / 3900 Correct Answers
1454 / 

In [25]:
import pandas as pd

df = pd.read_csv('llama_mmlu_responses_run5.csv')

print(f'Average: {float(df["Is Correct"].mean()) * 100}')

Average: 36.31249109813417


# Run 6: Compartmentalized prompting

In this run, prompt shots are dependent on the subject of the current question being asked. In all prior runs, prompts were dependent on randomly nominated examples of questions and answers regardless of the subject.

In [139]:
import csv
from tqdm.notebook import tqdm
from transformers import pipeline
import torch

# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your LLaMA model pipeline
pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype=torch.float16,  # Use FP16 for faster GPU performance
    device=0,  # Use the first GPU
    pad_token_id=tokenizer.eos_token_id
)

# Preprocessing
categories = train_dataset.unique('subject')
# print(len(categories))
items_compartmentalized = {}
for c in categories:
    items_compartmentalized[c] = {'shots': train_dataset.filter(lambda example: example['subject'] == c)}

few_shot_prompt = "You are a bot that's trained to answer questions in JSON format. For example:\n\n"

for c in categories:
    idx = 0
    per_subject_prompt = '' + few_shot_prompt
    for item in items_compartmentalized[c]['shots']:
        question = item['question']
        choices = item['choices']
        answer_index = item['answer']
        actual_answer = choices[answer_index]
        json_answer = {"Answer": f"{actual_answer}"}
        stringified_json_answer = re.sub('\'', '"', str(json_answer))
        item_prompt = f'Question: {question}\nChoices: {choices}\n{stringified_json_answer}'
        per_subject_prompt = per_subject_prompt + item_prompt
        if idx < 4:
            per_subject_prompt += '\n' + ('-' * 25) + '\n'
        idx += 1
    items_compartmentalized[c]['prompt'] = per_subject_prompt + "\n\nAfter here, you will be given questions. Select the best answer from the choices provided. Do not explain your answer. Format your answer as with the examples above."

In [140]:
print(items_compartmentalized['abstract_algebra']['prompt'])

You are a bot that's trained to answer questions in JSON format. For example:

Question: Find all c in Z_3 such that Z_3[x]/(x^2 + c) is a field.
Choices: ['0', '1', '2', '3']
{"Answer": "1"}
-------------------------
Question: Statement 1 | If aH is an element of a factor group, then |aH| divides |a|. Statement 2 | If H and K are subgroups of G then HK is a subgroup of G.
Choices: ['True, True', 'False, False', 'True, False', 'False, True']
{"Answer": "False, False"}
-------------------------
Question: Statement 1 | Every element of a group generates a cyclic subgroup of the group. Statement 2 | The symmetric group S_10 has 10 elements.
Choices: ['True, True', 'False, False', 'True, False', 'False, True']
{"Answer": "True, False"}
-------------------------
Question: Statement 1| Every function from a finite set onto itself must be one to one. Statement 2 | Every subgroup of an abelian group is abelian.
Choices: ['True, True', 'False, False', 'True, False', 'False, True']
{"Answer": "T

In [151]:
import json
import re
from tqdm.notebook import tqdm

rows = []
is_correct_count = 0
output_file = 'llama_mmlu_responses_run7.csv'

for idx in tqdm(range(0, len(dataset)), desc="Processing evaluation set"):
    format_error = 'None'
    example = dataset[idx]

    # Prepare the user question with few-shot prompt
    question = example['question']
    choices = example['choices']
    correct_answer = example['answer']
    subject = example['subject']
    
    stringified_choices = re.sub('"', '\'', str(choices))
    # Append the new question to the few-shot prompt
    user_question = f"Question: {question}\nChoices: {stringified_choices}\nWhat is your final answer?"
    
    # Messages to be passed to the model
    messages = [
        {'role': 'system', 'content': items_compartmentalized[subject]['prompt']},
        {'role': 'user', 'content': user_question},
    ]
    
    outputs = pipe(messages, max_new_tokens=20, temperature=0.1)
    
    output = outputs[0]['generated_text'][-1]['content']
    
    # Extract LLaMA's answer (post-processed)
    llama_answer = output.lower()
    try:
        llama_answer = json.loads(llama_answer)['answer']
    except Exception as e:
        format_error = 'Answer is not in JSON format.'
        is_correct = choices[correct_answer].lower() in re.sub('"', '\'', llama_answer).lower()
        is_correct_count += 1 if is_correct else 0
        rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, subject, format_error])
        if idx % 100 == 0:
            print(f'{is_correct_count} / {idx} Correct Answers')
        continue


    # Check if the answer matches the correct answer
    if type(llama_answer) == str:
        is_correct = choices[correct_answer].lower() == re.sub('"', '\'', llama_answer).lower()
    else:
        is_correct = False
        format_error = 'Parsed answer is still dictionary.'

    is_correct_count += 1 if is_correct else 0
    if idx % 100 == 0:
        print(f'{is_correct_count} / {idx} Correct Answers')
    # Collect the result row
    rows.append([question, choices, llama_answer, choices[correct_answer], is_correct, subject, format_error])

# Write the collected results to a CSV file
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct", "Subject", "Format Error"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")

Processing evaluation set:   0%|          | 0/14042 [00:00<?, ?it/s]

0 / 0 Correct Answers
18 / 100 Correct Answers
62 / 200 Correct Answers
99 / 300 Correct Answers
138 / 400 Correct Answers
189 / 500 Correct Answers
220 / 600 Correct Answers
261 / 700 Correct Answers
305 / 800 Correct Answers
347 / 900 Correct Answers
377 / 1000 Correct Answers
408 / 1100 Correct Answers
436 / 1200 Correct Answers
473 / 1300 Correct Answers
510 / 1400 Correct Answers
541 / 1500 Correct Answers
595 / 1600 Correct Answers
638 / 1700 Correct Answers
679 / 1800 Correct Answers
706 / 1900 Correct Answers
745 / 2000 Correct Answers
786 / 2100 Correct Answers
818 / 2200 Correct Answers
849 / 2300 Correct Answers
879 / 2400 Correct Answers
911 / 2500 Correct Answers
944 / 2600 Correct Answers
988 / 2700 Correct Answers
1030 / 2800 Correct Answers
1074 / 2900 Correct Answers
1111 / 3000 Correct Answers
1142 / 3100 Correct Answers
1185 / 3200 Correct Answers
1230 / 3300 Correct Answers
1266 / 3400 Correct Answers
1305 / 3500 Correct Answers
1351 / 3600 Correct Answers
1396 / 37

In [30]:
import pandas as pd 

df = pd.read_csv('llama_mmlu_responses_run6.csv')
class_accuracies = df.groupby('Subject')['Is Correct'].mean().reset_index()
print(class_accuracies)

macro_accuracy = class_accuracies['Is Correct'].mean()
print('Macro Accuracy', macro_accuracy * 100)
print(f'Micro Accuracy: {float(df["Is Correct"].mean()) * 100}')

                                Subject  Is Correct
0                      abstract_algebra    0.180000
1                               anatomy    0.459259
2                             astronomy    0.355263
3                       business_ethics    0.490000
4                    clinical_knowledge    0.381132
5                       college_biology    0.416667
6                     college_chemistry    0.330000
7              college_computer_science    0.300000
8                   college_mathematics    0.270000
9                      college_medicine    0.404624
10                      college_physics    0.225490
11                    computer_security    0.550000
12                   conceptual_physics    0.417021
13                         econometrics    0.289474
14               electrical_engineering    0.434483
15               elementary_mathematics    0.309524
16                         formal_logic    0.309524
17                         global_facts    0.380000
18          

In [158]:
print(len(items_compartmentalized.items()))

57


# Run 7: Use Prompts from Official LLaMA Eval

From their [eval details](https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/eval_details.md),

> Macro averages are reported unless otherwise stated. The micro average scores for the various models are: 65.6, 79.0, and 85.4 for the pre-trained 8B, 70B and 405B models respectively for the 5-shot config; 69.44, 84.0, 87.71 for the post-trained 8B, 70B and 405B models respectively for the 5-shot config.

In [36]:
from tqdm.notebook import tqdm
from transformers import pipeline
import torch

# Load your dataset and set up the model pipeline
# output_file = 'llama_mmlu_responses_5shot.csv'

# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your LLaMA model pipeline
pipe = pipeline(
    'text-generation',
    model=model_path,
    torch_dtype=torch.float16,  # Use FP16 for faster GPU performance
    device=0,  # Use the first GPU
    pad_token_id=tokenizer.eos_token_id
)

import pandas as pd

# These are the details coming from HuggingFace evaluators
# https://huggingface.co/datasets/meta-llama/Llama-3.2-1B-Instruct-evals/tree/main/Llama-3.2-1B-Instruct-evals
data = pd.read_parquet("Details_mmlu_2024-09-23T17-23-07.347337.parquet.gzip")
print(data.count()) 

task_type                                 14042
task_name                                 14042
subtask_name                              14042
input_question                            14042
input_choice_list                         14042
input_final_prompts                       14042
input_correct_responses                   14042
output_prediction_text                    14042
output_parsed_answer                      14042
output_choice_completions                     0
output_choice_negative_log_likelihoods        0
output_metrics                            14042
is_correct                                14042
input_question_hash                       14042
input_final_prompts_hash                  14042
benchmark_label                           14042
eval_config                               14042
dtype: int64


These are the optimized settings mentioned by the evaluators for the official eval. https://huggingface.co/datasets/meta-llama/Llama-3.2-1B-Instruct-evals/tree/main/Llama-3.2-1B-Instruct-evals 

In [51]:
# This file can be found in: https://huggingface.co/datasets/meta-llama/Llama-3.2-1B-Instruct-evals/tree/main/Llama-3.2-1B-Instruct-evals
metrics = pd.read_parquet('Llama-3.2-1B-Instruct-evals_Details_metrics_details_2024-09-23T17-23-22.207810.parquet.gzip')
metrics[metrics['benchmark_label'] == 'MMLU'].iloc[0]['eval_config']

{'top_p': '0',
 'seed': '42',
 'num_generations': '1',
 'max_gen_len': '10',
 'top_k': '0',
 'temperature': '0.0',
 'prompt_fn': 'functools.partial(<function jinja_dialog_format at 0x7f02e368d090>, template={\'prompt\': \'Given the following question and four candidate answers (A, B, C and D), choose the best answer.\\nQuestion: {{ question }}\\nA. {{ choices["A"] }}\\nB. {{ choices["B"] }}\\nC. {{ choices["C"] }}\\nD. {{ choices["D"] }}\\nYour response should end with "The best answer is [the_answer_letter]" where the [the_answer_letter] is one of A, B, C or D.\\n\', \'answer\': \'The best answer is {{ answer }}.\', \'gen_prefix\': \'The best answer is \'}, append_gen_prefix=True)',
 'num_few_shot': '5',
 'max_prompt_len': '3840',
 'return_logprobs': 'false'}

In [40]:
# Example prompt format
print(data.iloc[0]['input_final_prompts'][0])

<|start_header_id|>user<|end_header_id|>

Given the following question and four candidate answers (A, B, C and D), choose the best answer.
Question: The energy for all forms of muscle contraction is provided by:
A. ATP.
B. ADP.
C. phosphocreatine.
D. oxidative phosphorylation.
Your response should end with "The best answer is [the_answer_letter]" where the [the_answer_letter] is one of A, B, C or D.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The best answer is A.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following question and four candidate answers (A, B, C and D), choose the best answer.
Question: What is the difference between a male and a female catheter?
A. Male and female catheters are different colours.
B. Male catheters are longer than female catheters.
C. Male catheters are bigger than female catheters.
D. Female catheters are longer than male catheters.
Your response should end with "The best answer is [the_answer_letter]" where the [the_answer

In [5]:
import csv
import re
from tqdm.notebook import tqdm

output_file = f'llama_mmlu_responses_run7.csv'

def parse_answer(answer_string):
    # Check if the string starts with the expected phrase
    expected_start = "The best answer is "
    answer_string = re.sub(r'\\n', '', answer_string.strip())
    
    # Guard clause to handle cases where the string doesn't start as expected
    if not answer_string.startswith(expected_start):
        raise ValueError(f"Invalid answer format: '{answer_string}' does not start with '{expected_start}'")

    # Parse the answer part (assuming it's a single character like 'A', 'B', etc.)
    answer = answer_string[len(expected_start):len(expected_start)+1].strip()

    # Additional check to ensure there's an answer after the phrase
    if not answer or len(answer) > 1:
        raise ValueError(f"Invalid or missing answer choice in '{answer_string}'")
    
    return answer


total = 0
correct_count = 0
rows = []
output_file = 'llama_mmlu_responses_run7.csv'

for i in tqdm(range(len(data))):
    total += 1
    item = data.iloc[i]
    final_prompts = item['input_final_prompts']
    correct_answer = item['input_correct_responses'][0]
    question = item['input_question']
    choices = item['input_choice_list']
    subject = item['subtask_name']

    messages = [
        {'role': 'system', 'content': final_prompts},
    ]

    outputs = pipe(messages, max_new_tokens=10, do_sample=False)
    
    output = outputs[0]['generated_text'][-1]['content'].strip()
    llama_answer = ''

    try:
        llama_answer = parse_answer(output)
    except ValueError:
        llama_answer = f'Invalid string format: {output}'
    
    is_correct = False
    if correct_answer == llama_answer:
        correct_count += 1
        is_correct = True

    if i % 100 == 0:
        print(f'{correct_count} / {i} Correct Answers')
    rows.append([question, choices, llama_answer, correct_answer, is_correct, subject])

with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Question", "Choices", "LLaMA Answer", "Correct Answer", "Is Correct", "Subject"])
    # Write all rows at once
    writer.writerows(rows)

print(f"Results saved to {output_file}")

  0%|          | 0/14042 [00:00<?, ?it/s]

/data/students/juan/anaconda3/envs/dl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data/students/juan/anaconda3/envs/dl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/data/students/juan/anaconda3/envs/dl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, th

0 / 0 Correct Answers


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


51 / 100 Correct Answers
106 / 200 Correct Answers
157 / 300 Correct Answers
181 / 400 Correct Answers
214 / 500 Correct Answers
267 / 600 Correct Answers
309 / 700 Correct Answers
363 / 800 Correct Answers
395 / 900 Correct Answers
443 / 1000 Correct Answers
491 / 1100 Correct Answers
519 / 1200 Correct Answers
534 / 1300 Correct Answers
572 / 1400 Correct Answers
599 / 1500 Correct Answers
609 / 1600 Correct Answers
647 / 1700 Correct Answers
720 / 1800 Correct Answers
754 / 1900 Correct Answers
785 / 2000 Correct Answers
827 / 2100 Correct Answers
865 / 2200 Correct Answers
910 / 2300 Correct Answers
959 / 2400 Correct Answers
997 / 2500 Correct Answers
1026 / 2600 Correct Answers
1063 / 2700 Correct Answers
1091 / 2800 Correct Answers
1147 / 2900 Correct Answers
1189 / 3000 Correct Answers
1225 / 3100 Correct Answers
1258 / 3200 Correct Answers
1314 / 3300 Correct Answers
1382 / 3400 Correct Answers
1448 / 3500 Correct Answers
1503 / 3600 Correct Answers
1553 / 3700 Correct Answers

In [44]:
import pandas as pd 

df = pd.read_csv('llama_mmlu_responses_run7.csv')
class_accuracies = df.groupby('Subject')['Is Correct'].mean().reset_index()
print(class_accuracies)

macro_accuracy = class_accuracies['Is Correct'].mean()
print('Macro Accuracy', macro_accuracy * 100)
print(f'Micro Accuracy: {float(df["Is Correct"].mean()) * 100}')


                                          Subject  Is Correct
0                      mmlu_chat.abstract_algebra    0.290000
1                               mmlu_chat.anatomy    0.496296
2                             mmlu_chat.astronomy    0.559211
3                       mmlu_chat.business_ethics    0.450000
4                    mmlu_chat.clinical_knowledge    0.471698
5                       mmlu_chat.college_biology    0.500000
6                     mmlu_chat.college_chemistry    0.350000
7              mmlu_chat.college_computer_science    0.330000
8                   mmlu_chat.college_mathematics    0.330000
9                      mmlu_chat.college_medicine    0.421965
10                      mmlu_chat.college_physics    0.196078
11                    mmlu_chat.computer_security    0.570000
12                   mmlu_chat.conceptual_physics    0.400000
13                         mmlu_chat.econometrics    0.263158
14               mmlu_chat.electrical_engineering    0.565517
15      

In [53]:
phrase = 'Invalid string format'
df['contains_phrase'] = df['LLaMA Answer'].str.contains(phrase, case=False, na=False)
grouped = df.groupby('Subject')
percentages = ((grouped['contains_phrase'].count() - grouped['contains_phrase'].sum()) / grouped['contains_phrase'].count()) * 100

print(percentages)

Subject
mmlu_chat.abstract_algebra                       100.000000
mmlu_chat.anatomy                                100.000000
mmlu_chat.astronomy                               98.684211
mmlu_chat.business_ethics                        100.000000
mmlu_chat.clinical_knowledge                      95.094340
mmlu_chat.college_biology                        100.000000
mmlu_chat.college_chemistry                       99.000000
mmlu_chat.college_computer_science               100.000000
mmlu_chat.college_mathematics                    100.000000
mmlu_chat.college_medicine                        98.265896
mmlu_chat.college_physics                        100.000000
mmlu_chat.computer_security                       99.000000
mmlu_chat.conceptual_physics                     100.000000
mmlu_chat.econometrics                           100.000000
mmlu_chat.electrical_engineering                 100.000000
mmlu_chat.elementary_mathematics                 100.000000
mmlu_chat.formal_logic          

In [65]:
metrics_df = metrics[metrics['benchmark_label'] == 'MMLU']
phrase = 'correct_format'
# metrics_df
metrics_df['contains_phrase'] = metrics_df['metric_tag'].str.contains(phrase, case=False, na=False)
metrics_df[metrics_df['contains_phrase'] == True][['metric_tag', 'metric_value_computed']]
# metrics_df


/tmp/ipykernel_2056250/3806437871.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics_df['contains_phrase'] = metrics_df['metric_tag'].str.contains(phrase, case=False, na=False)


,metric_tag,metric_value_computed
890,mmlu_chat.world_religions/correct_format,100.0
892,mmlu_chat.virology/correct_format,100.0
894,mmlu_chat.us_foreign_policy/correct_format,100.0
896,mmlu_chat.sociology/correct_format,100.0
898,mmlu_chat.security_studies/correct_format,100.0
900,mmlu_chat.public_relations/correct_format,100.0
902,mmlu_chat.professional_psychology/correct_format,100.0
904,mmlu_chat.professional_medicine/correct_format,100.0
906,mmlu_chat.professional_law/correct_format,100.0
908,mmlu_chat.professional_accounting/correct_format,100.0
